In [1]:
import pandas as pd
import sys
sys.path.append("/root/workSpace/investProject/ConvertBondWheel")

In [2]:
from policy.double_low_enhance.alarm import *

/root/workSpace/investProject/ConvertBondWheel


In [7]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 5000)
pd.set_option('float_format', lambda x: '%.2f' % x)
logging.basicConfig(level=logging.INFO)

In [31]:
#对应配置 /root/workSpace/investProject/ConvertBondWheel/policy/double_low_enhance/config.py
double_low_conf = {
    'db_topk': 0.1,
    'min_curr_iss_amt': 9,  # <剩余规模
    'volatility': 0.1,  # >波动率
    'premium_rt': 0.2,  # <溢价率
    'min_price': 115,  # <可转债价格
    'min_dblow': 125,  # <双低价格
    'redeem': ['Y'],  #  Y=强赎 不在强赎
    'min_year_left': 1,  # >剩余到期年限 (不要一年内到期的)
    'ytm_rt': -0.01,  # > 到期收益率
    'pluse_out_step':10, # 脉冲轮出时， 轮出债比轮入新债双低值大多少
    'pulse_out_price_median':110, # 脉冲轮出时， 市场价格中位数
    'pulse_out_price':[120, 125], # 脉冲轮出时< 110 和>110 对应的轮出价格
    'pulse_out_dblow':[125, 130],  #同上， 双低值
    'user_zh_columns':[u'转债名称', u'双低', u'现价', u'溢价率', u'剩余规模', u'波动率', u'涨跌幅', u'成交额(万元)', u'换手率'],
} 

#user_zh_columns
# 展示字段， 可以继续添加，
# 如果报错， 说明没有爬取到该字段， 可以通过以下步骤来新增：
# 1. vim /root/workSpace/investProject/ConvertBondWheel/ConvertBondWheel/fields_config.py
# jisilu_dict 新增对应key-value, jisulu_used_columns 新增对应英文名
# 2. vim /root/workSpace/investProject/ConvertBondWheel/ConvertBondWheel/items.py
# 添加  该字段 = scrapy.Field()

In [32]:
# 读当前可转债数据
bound = read_data()
balance = read_balancing()

#轮动策略
in_bound = periodWheel(bound,double_low_conf)

# 结果换成中文header & 按列排序
result = display(bound, balance, in_bound, None, user_zh_columns=double_low_conf['user_zh_columns'])
result[:10]

,转债名称,双低,现价,溢价率,剩余规模,波动率,涨跌幅,成交额(万元),换手率,代码,正股名称,正股价,正股涨跌,PB,转股价,转股价值,评价,回售触发价,强赎触发价,转债占比,转股起始日,到期时间,剩余年限,到期税前收益,到期税后收益,强赎公告
0,*文科转债,109.64,99.40,10.24%,9.50,28.04%,0.17%,1766.84,1.87%,128127,文科园林,4.40,-0.23%,0.91,4.88,90.16,AA-,3.42,6.34,48.50%,2021-03-01,26-08-19,5.21,3.99%,3.25%,X
1,*靖远转债,110.39,99.89,10.50%,28.00,45.26%,-0.65%,4236.47,1.51%,127027,靖远煤电,2.92,-2.01%,0.79,3.23,90.40,AA+,2.26,4.20,42.00%,2021-06-16,26-12-09,5.52,2.69%,2.17%,X
2,*东湖转债,110.79,104.36,6.43%,15.50,31.28%,0.01%,1942.94,1.20%,110080,东湖高新,6.04,-0.33%,0.89,6.16,98.05,AA,4.31,8.01,33.10%,2021-10-18,27-04-12,5.86,1.72%,1.24%,X
3,洪涛转债,111.48,105.97,5.51%,11.59,29.95%,-0.32%,1077.37,0.88%,128013,洪涛股份,2.33,-1.27%,1.04,2.32,100.43,AA-,1.62,3.02,52.30%,2017-02-06,22-07-29,1.15,3.18%,1.55%,X
4,*中装转2,112.59,101.10,11.49%,11.60,39.85%,0.03%,3644.56,3.11%,127033,中装建设,5.74,-0.52%,1.23,6.33,90.68,AA,4.43,8.23,31.70%,2021-10-22,27-04-15,5.87,2.58%,2.05%,X
5,*孚日转债,113.16,103.41,9.75%,6.50,32.15%,-0.18%,674.67,1.00%,128087,孚日股份,4.24,-0.24%,1.06,4.50,94.22,AA,3.15,5.85,16.90%,2020-06-23,25-12-17,4.54,1.99%,1.46%,X
6,*灵康转债,114.75,108.73,6.02%,5.25,38.27%,0.63%,2071.91,3.63%,113610,灵康药业,8.83,1.73%,4.16,8.61,102.56,AA-,6.03,11.19,8.30%,2021-06-07,26-11-30,5.49,2.02%,1.35%,X
7,海亮转债,115.73,110.40,5.33%,30.06,29.35%,-0.68%,2065.80,0.62%,128081,海亮股份,10.23,-0.39%,1.98,9.76,104.82,AA,6.83,12.69,15.90%,2020-05-27,25-11-21,4.47,0.92%,0.31%,X
8,长集转债,115.87,108.00,7.87%,8.00,40.56%,-8.39%,169192.60,189.28%,128105,长青集团,7.92,4.21%,2.10,7.91,100.13,AA,5.54,10.28,21.50%,2020-10-15,26-04-08,4.85,1.31%,0.75%,X
9,*鸿达转债,116.29,116.78,-0.49%,8.41,48.04%,3.25%,36515.70,37.63%,128085,鸿达兴业,4.60,3.84%,1.55,3.92,117.35,BB,2.74,5.10,6.30%,2020-06-22,25-12-16,4.54,1.44%,0.52%,X


In [26]:
result.describe()

,双低,现价,剩余规模,成交额(万元),代码,正股价,PB,转股价,转股价值,强赎触发价,剩余年限
count,363.00,363.00,363.00,363.00,363.00,363.00,363.00,363.00,363.00,363.00,363.00
mean,157.23,127.27,16.00,13797.35,120615.18,17.62,2.86,16.94,103.45,22.01,4.51
std,84.40,72.50,44.61,50103.89,6770.05,24.97,2.38,18.86,71.92,24.53,1.12
min,109.64,80.37,0.33,27.10,110033.00,1.65,0.44,2.32,26.09,3.02,0.59
25%,128.22,104.75,3.00,595.10,113564.00,7.22,1.46,7.72,78.83,10.02,3.83
50%,138.43,112.47,6.00,1693.33,123057.00,11.29,2.13,12.20,93.54,15.86,4.85
75%,154.30,125.85,11.54,6245.83,128021.50,18.62,3.25,19.11,111.90,24.84,5.38
max,1295.15,1294.00,499.99,659378.81,128145.00,286.30,19.60,222.83,1279.33,289.68,5.90


In [34]:
! sh /root/workSpace/investProject/ConvertBondWheel/policy/double_low_enhance/run_alarm.sh

---------2021-06-06 14:19:43------
---------scrapy---------
2021-06-06 14:19:44 [scrapy.utils.log] INFO: Scrapy 2.4.0 started (bot: ConvertBondWheel)
2021-06-06 14:19:44 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 20.3.0, Python 3.8.5 (default, Sep  4 2020, 07:30:14) - [GCC 7.3.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.1.1, Platform Linux-3.10.0-862.el7.x86_64-x86_64-with-glibc2.10
2021-06-06 14:19:44 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'ConvertBondWheel',
 'CONCURRENT_REQUESTS': 64,
 'CONCURRENT_REQUESTS_PER_DOMAIN': 64,
 'CONCURRENT_REQUESTS_PER_IP': 64,
 'LOG_LEVEL': 'INFO',
 'NEWSPIDER_MODULE': 'ConvertBondWheel.spiders',
 'SPIDER_MODULES': ['ConvertBondWheel.spiders']}
2021-06-06 14:19:44 [scrapy.extensions.telnet] INFO: Telnet Password: b298641efc1c1ea6
2021-06-06 14:19:44 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 '

--------alarm-----------
/root/workSpace/investProject/ConvertBondWheel
INFO:root:[/root/workSpace/investProject/ConvertBondWheel/policy/double_low_enhance/alarm.py pulse start!]
DEBUG:root:Read bound data:
DEBUG:root:      dblow  bond_id bond_nm  ...  curr_iss_amt  redeem_flag turnover_rt
0    109.65   128127    文科转债  ...         9.498            X        1.87
1    110.39   127027    靖远转债  ...        28.000            X        1.51
2    110.80   110080    东湖转债  ...        15.500            X        1.20
3    111.49   128013    洪涛转债  ...        11.590            X        0.88
4    112.59   127033    中装转2  ...        11.600            X        3.11
..      ...      ...     ...  ...           ...          ...         ...
358  435.99   113580    康隆转债  ...         0.411            N      275.90
359  436.43   110044    广电转债  ...         0.709            N       33.99
360  482.36   123015    蓝盾转债  ...         1.003            X       58.23
361  516.00   128041    盛路转债  ...         0.530     